# 🎯 QML Pattern Visual Lab

Interactive Jupyter notebook for perfecting QML pattern visualizations.

**Usage:** Change `target_id` in Cell 3 and press `Shift+Enter` to see each pattern.

In [41]:
# =============================================================================
# CELL 1: DATA LOADING
# =============================================================================

import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Enable inline plotting
%matplotlib inline

# Import ccxt for OHLCV data
import ccxt

# =============================================================================
# LOAD PATTERN DATA (v1.1.0 Rolling Window Backtest)
# =============================================================================

PATTERNS_CSV = Path('./btc_backtest_labels.csv')
patterns_df = pd.read_csv(PATTERNS_CSV, parse_dates=['time'])
print(f"✅ Loaded {len(patterns_df)} patterns from v1.1.0 rolling-window backtest")
print(f"   Date range: {patterns_df['time'].min()} to {patterns_df['time'].max()}")

# =============================================================================
# FETCH AND CACHE OHLCV DATA
# =============================================================================

print("\n📡 Fetching BTC/USDT 1h OHLCV data...")

exchange = ccxt.binance({'enableRateLimit': True})

# Fetch data covering all patterns with padding
start_date = patterns_df['time'].min() - timedelta(days=10)
end_date = patterns_df['time'].max() + timedelta(days=5)

start_ts = int(start_date.timestamp() * 1000)
end_ts = int(end_date.timestamp() * 1000)

all_candles = []
current_ts = start_ts

while current_ts < end_ts:
    ohlcv = exchange.fetch_ohlcv('BTC/USDT', '1h', since=current_ts, limit=1000)
    if not ohlcv:
        break
    all_candles.extend(ohlcv)
    last_ts = ohlcv[-1][0]
    if last_ts <= current_ts:
        break
    current_ts = last_ts + 1

ohlcv_df = pd.DataFrame(all_candles, columns=['time', 'Open', 'High', 'Low', 'Close', 'Volume'])
ohlcv_df['time'] = pd.to_datetime(ohlcv_df['time'], unit='ms')
ohlcv_df = ohlcv_df.drop_duplicates(subset=['time']).sort_values('time')
ohlcv_df = ohlcv_df.set_index('time')

print(f"✅ Loaded {len(ohlcv_df)} candles: {ohlcv_df.index.min()} to {ohlcv_df.index.max()}")

# Display pattern summary
patterns_df[['time', 'pattern_type', 'validity_score', 'entry_price', 'stop_loss']].head(10)

ModuleNotFoundError: No module named 'mplfinance'

In [ ]:
# =============================================================================
# CELL 2: THE VISUALIZATION FUNCTION
# =============================================================================

def find_swing_points(df, window=5):
    """Find swing highs and lows."""
    highs, lows = [], []
    for i in range(window, len(df) - window):
        if df['High'].iloc[i] == df['High'].iloc[i-window:i+window+1].max():
            highs.append((df.index[i], df['High'].iloc[i]))
        if df['Low'].iloc[i] == df['Low'].iloc[i-window:i+window+1].min():
            lows.append((df.index[i], df['Low'].iloc[i]))
    return highs, lows


def reconstruct_points(df, pattern):
    """Reconstruct QML 5-point structure."""
    pattern_time = pd.Timestamp(pattern['time'])
    is_bullish = 'bullish' in pattern['pattern_type']
    
    head_price = pattern['head_price']
    ls_price = pattern['left_shoulder_price']
    entry_price = pattern['entry_price']
    
    # Find pattern in data
    idx = df.index.get_indexer([pattern_time], method='nearest')[0]
    lookback = min(100, idx)
    window_df = df.iloc[max(0, idx - lookback):idx + 10]
    
    if len(window_df) < 20:
        return None
    
    swing_highs, swing_lows = find_swing_points(window_df, window=3)
    
    if is_bullish:
        # P3 (Head) - swing low
        candidates = [(t, p) for t, p in swing_lows if t < pattern_time and abs(p - head_price) / head_price < 0.02]
        if candidates:
            p3_time, p3_price = min(candidates, key=lambda x: abs(x[1] - head_price))
        else:
            p3_time = window_df['Low'].idxmin()
            p3_price = window_df['Low'].min()
        
        # P2 (LS) - swing high before head
        candidates = [(t, p) for t, p in swing_highs if t < p3_time]
        if candidates:
            p2_time, p2_price = max(candidates, key=lambda x: x[0])
        else:
            pre = window_df[window_df.index < p3_time]
            p2_time = pre['High'].idxmax() if len(pre) > 0 else p3_time
            p2_price = pre['High'].max() if len(pre) > 0 else head_price
        
        # P1 (Base) - swing low before LS
        candidates = [(t, p) for t, p in swing_lows if t < p2_time]
        if candidates:
            p1_time, p1_price = max(candidates, key=lambda x: x[0])
        else:
            pre = window_df[window_df.index < p2_time]
            p1_time = pre['Low'].idxmin() if len(pre) > 0 else p2_time
            p1_price = pre['Low'].min() if len(pre) > 0 else ls_price
        
        # Trend start (before P1)
        pre_p1 = window_df[window_df.index < p1_time]
        if len(pre_p1) > 10:
            trend_start_time = pre_p1.index[0]
            trend_start_price = pre_p1['High'].max()
        else:
            trend_start_time = p1_time - timedelta(hours=20)
            trend_start_price = p2_price * 1.02
        
        # P4 (CHoCH) - swing high after head
        candidates = [(t, p) for t, p in swing_highs if t > p3_time and t < pattern_time]
        if candidates:
            p4_time, p4_price = min(candidates, key=lambda x: x[0])
        else:
            post = window_df[(window_df.index > p3_time) & (window_df.index <= pattern_time)]
            p4_time = post['High'].idxmax() if len(post) > 0 else pattern_time
            p4_price = post['High'].max() if len(post) > 0 else entry_price
        
        p5_time, p5_price = pattern_time, entry_price
        
    else:  # Bearish
        # P3 (Head) - swing high
        candidates = [(t, p) for t, p in swing_highs if t < pattern_time and abs(p - head_price) / head_price < 0.02]
        if candidates:
            p3_time, p3_price = min(candidates, key=lambda x: abs(x[1] - head_price))
        else:
            p3_time = window_df['High'].idxmax()
            p3_price = window_df['High'].max()
        
        # P2 (LS) - swing low before head
        candidates = [(t, p) for t, p in swing_lows if t < p3_time]
        if candidates:
            p2_time, p2_price = max(candidates, key=lambda x: x[0])
        else:
            pre = window_df[window_df.index < p3_time]
            p2_time = pre['Low'].idxmin() if len(pre) > 0 else p3_time
            p2_price = pre['Low'].min() if len(pre) > 0 else head_price
        
        # P1 (Base) - swing high before LS
        candidates = [(t, p) for t, p in swing_highs if t < p2_time]
        if candidates:
            p1_time, p1_price = max(candidates, key=lambda x: x[0])
        else:
            pre = window_df[window_df.index < p2_time]
            p1_time = pre['High'].idxmax() if len(pre) > 0 else p2_time
            p1_price = pre['High'].max() if len(pre) > 0 else ls_price
        
        # Trend start
        pre_p1 = window_df[window_df.index < p1_time]
        if len(pre_p1) > 10:
            trend_start_time = pre_p1.index[0]
            trend_start_price = pre_p1['Low'].min()
        else:
            trend_start_time = p1_time - timedelta(hours=20)
            trend_start_price = p2_price * 0.98
        
        # P4 (CHoCH) - swing low after head
        candidates = [(t, p) for t, p in swing_lows if t > p3_time and t < pattern_time]
        if candidates:
            p4_time, p4_price = min(candidates, key=lambda x: x[0])
        else:
            post = window_df[(window_df.index > p3_time) & (window_df.index <= pattern_time)]
            p4_time = post['Low'].idxmin() if len(post) > 0 else pattern_time
            p4_price = post['Low'].min() if len(post) > 0 else entry_price
        
        p5_time, p5_price = pattern_time, entry_price
    
    return {
        'trend_start': (trend_start_time, trend_start_price),
        'P1': (p1_time, p1_price),
        'P2': (p2_time, p2_price),
        'P3': (p3_time, p3_price),
        'P4': (p4_time, p4_price),
        'P5': (p5_time, p5_price),
        'is_bullish': is_bullish,
    }


def plot_pattern(pattern_id):
    """
    Plot a QML pattern with TradingView-quality visualization.
    
    Args:
        pattern_id: Index of pattern in patterns_df (0-39)
    """
    if pattern_id < 0 or pattern_id >= len(patterns_df):
        print(f"❌ Invalid pattern_id. Must be 0-{len(patterns_df)-1}")
        return
    
    pattern = patterns_df.iloc[pattern_id]
    print(f"\n{'='*70}")
    print(f"📊 Pattern #{pattern_id}: {pattern['pattern_type'].upper()}")
    print(f"   Time: {pattern['time']}")
    print(f"   Validity: {pattern['validity_score']:.3f}")
    print(f"   Entry: ${pattern['entry_price']:,.2f} | SL: ${pattern['stop_loss']:,.2f}")
    print(f"{'='*70}")
    
    # Reconstruct points
    points = reconstruct_points(ohlcv_df, pattern)
    if not points:
        print("❌ Could not reconstruct pattern points")
        return
    
    # Get chart window with padding (40-50 bars left, 20-30 right)
    p1_time = points['P1'][0]
    p5_time = points['P5'][0]
    
    p1_idx = ohlcv_df.index.get_indexer([p1_time], method='nearest')[0]
    p5_idx = ohlcv_df.index.get_indexer([p5_time], method='nearest')[0]
    
    start_idx = max(0, p1_idx - 45)  # 40-50 bars padding left
    end_idx = min(len(ohlcv_df) - 1, p5_idx + 25)  # 20-30 bars padding right
    
    chart_df = ohlcv_df.iloc[start_idx:end_idx + 1].copy()
    
    # ==========================================================================
    # BUILD ALINES (Arbitrary Lines)
    # ==========================================================================
    
    # 1. Trend line (GRAY) - from trend start to P1
    trend_line = [
        (points['trend_start'][0], points['trend_start'][1]),
        (points['P1'][0], points['P1'][1]),
    ]
    
    # 2. QML Structure line (BLUE) - P1 → P2 → P3 → P4 → P5
    qml_line = [
        (points['P1'][0], points['P1'][1]),
        (points['P2'][0], points['P2'][1]),
        (points['P3'][0], points['P3'][1]),
        (points['P4'][0], points['P4'][1]),
        (points['P5'][0], points['P5'][1]),
    ]
    
    alines = [trend_line, qml_line]
    aline_colors = ['gray', '#2962ff']  # Gray for trend, Blue for QML
    
    # ==========================================================================
    # BUILD TRADE ZONES (Rectangle approximation using hlines)
    # ==========================================================================
    
    entry = pattern['entry_price']
    sl = pattern['stop_loss']
    tp = pattern['take_profit']
    is_bullish = points['is_bullish']
    
    # Create horizontal lines for SL and TP zones
    hlines_dict = dict(
        hlines=[sl, tp],
        colors=['#ef5350', '#26a69a'],  # Red for SL, Green for TP
        linestyle='--',
        linewidths=1.5,
    )
    
    # ==========================================================================
    # PLOT WITH LIGHT THEME
    # ==========================================================================
    
    # Custom light style similar to Yahoo
    mc = mpf.make_marketcolors(
        up='#26a69a',
        down='#ef5350',
        edge='inherit',
        wick='inherit',
    )
    style = mpf.make_mpf_style(
        base_mpf_style='yahoo',
        marketcolors=mc,
        gridstyle='-',
        gridcolor='#e0e0e0',
    )
    
    title = f"QML {pattern['pattern_type'].upper()} | {pattern['time'].strftime('%Y-%m-%d %H:%M')} | Validity: {pattern['validity_score']:.2f}"
    
    fig, axes = mpf.plot(
        chart_df,
        type='candle',
        style=style,
        title=title,
        ylabel='Price (USDT)',
        volume=False,
        figsize=(16, 9),
        alines=dict(alines=alines, colors=aline_colors, linewidths=[1.5, 2.5]),
        hlines=hlines_dict,
        returnfig=True,
        tight_layout=True,
    )
    
    ax = axes[0]
    
    # ==========================================================================
    # ADD LABELS (LS, H, LL, RS)
    # ==========================================================================
    
    labels = {'P2': 'LS', 'P3': 'H', 'P4': 'LL', 'P5': 'RS'}
    
    for pt_name, label in labels.items():
        pt_time, pt_price = points[pt_name]
        
        # Get x position
        if pt_time in chart_df.index:
            x_pos = list(chart_df.index).index(pt_time)
        else:
            x_pos = chart_df.index.get_indexer([pt_time], method='nearest')[0]
        
        # Offset based on point type
        if is_bullish:
            if pt_name in ['P2', 'P4']:  # highs
                y_off = pt_price * 1.008
                va = 'bottom'
            else:  # lows
                y_off = pt_price * 0.992
                va = 'top'
        else:
            if pt_name in ['P2', 'P4']:  # lows
                y_off = pt_price * 0.992
                va = 'top'
            else:  # highs
                y_off = pt_price * 1.008
                va = 'bottom'
        
        ax.annotate(
            label,
            xy=(x_pos, pt_price),
            xytext=(x_pos, y_off),
            fontsize=11,
            fontweight='bold',
            color='#2962ff',
            ha='center',
            va=va,
            bbox=dict(boxstyle='round,pad=0.2', facecolor='white', edgecolor='#2962ff', alpha=0.9),
        )
    
    # Add entry info box
    info = f"Entry: ${entry:,.2f}\nSL: ${sl:,.2f}\nTP: ${tp:,.2f}"
    ax.text(
        0.98, 0.02, info,
        transform=ax.transAxes,
        fontsize=10,
        verticalalignment='bottom',
        horizontalalignment='right',
        family='monospace',
        bbox=dict(boxstyle='round,pad=0.4', facecolor='white', edgecolor='#888', alpha=0.95),
    )
    
    plt.show()
    
    # Print point coordinates
    print("\n📍 Pattern Points:")
    for name in ['P1', 'P2', 'P3', 'P4', 'P5']:
        t, p = points[name]
        label = {'P1': 'Base', 'P2': 'LS', 'P3': 'Head', 'P4': 'LL', 'P5': 'RS'}[name]
        print(f"   {name} ({label}): {t.strftime('%Y-%m-%d %H:%M')} @ ${p:,.2f}")


print("✅ plot_pattern() function ready!")
print("   Usage: plot_pattern(0) to plot first pattern")
print(f"   Available patterns: 0 to {len(patterns_df)-1}")

In [ ]:
# =============================================================================
# CELL 3: EXECUTION
# =============================================================================
# Change this number and press Shift+Enter to view different patterns

target_id = 0  # Pattern index (0-39)

plot_pattern(target_id)

In [ ]:
# =============================================================================
# QUICK NAVIGATION: High-validity patterns
# =============================================================================

# Show patterns sorted by validity score
print("🏆 Patterns sorted by validity score:")
for i, row in patterns_df.nlargest(10, 'validity_score').iterrows():
    print(f"   ID={i}: {row['pattern_type']:12} | {row['time'].strftime('%Y-%m-%d')} | validity={row['validity_score']:.3f}")